In [38]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


In [52]:
data = pd.read_csv('titanic.csv')


In [53]:

pclass_1_mean_age = data.groupby(['Pclass']).mean().iloc[0]['Age']
pclass_2_mean_age = data.groupby(['Pclass']).mean().iloc[1]['Age']
pclass_3_mean_age = data.groupby(['Pclass']).mean().iloc[2]['Age']

In [54]:

def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return pclass_1_mean_age
        elif Pclass == 2:
            return pclass_2_mean_age
        else:
            return pclass_3_mean_age
    else:
        return Age

In [55]:
data['Age'] = data[['Age','Pclass']].apply(impute_age,axis=1)


In [56]:
data.drop('Cabin',axis=1,inplace=True)
data.dropna(inplace=True)

In [58]:
sex = pd.get_dummies(data['Sex'],drop_first=True)
embark = pd.get_dummies(data['Embarked'],drop_first=True)
data = pd.concat([data,sex,embark],axis=1)
data.drop(['Sex','Embarked','Name','Ticket','PassengerId'],axis=1,inplace=True)
data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,0,3,22.0,1,0,7.2500,1,0,1
1,1,1,38.0,1,0,71.2833,0,0,0
2,1,3,26.0,0,0,7.9250,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,1


In [59]:
y = data['Survived'].copy().as_matrix()


/Users/mengyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [60]:
X = data.drop('Survived',axis=1).as_matrix()


/Users/mengyu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [87]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4)


In [70]:

def sigmoid(z):
    return float(1 / float((1 + math.exp(-1 * z))))



In [71]:
def hypothesis(theta,x):
    z = 0
    for i in range(len(theta)):
        z += theta[i] * x[i]s
    return sigmoid(z)



In [79]:
def cost_function(X,y,theta,m):
    sumError = 0
    for i in range(m):
        error = 0
        x_i = X[i]
        y_i = y[i]
        h_i = hypothesis(theta,x_i)
        error = y_i * math.log(h_i) + (1 - y_i) * math.log(1 - h_i)
        sumError += error
    const = -1/m
    cost = const * sumError
    return cost


In [80]:
def cfd(X,y,theta,j,m):
    sumError = 0
    for i in range(m):
        x_i = X[i]
        x_ij = x_i[j]
        h_i = hypothesis(theta, x_i)
        error = (h_i - y[i]) * x_ij
        sumError += error
    const = 1/m
    cost = const * sumError
    return cost


In [81]:

def gradient_descent(X,y,theta,alpha,m):
    opt_theta = []
    for j in range(len(theta)):
        cost = cfd(X,y,theta,j,m)
        updated_theta = theta[j] - (alpha * cost)
        opt_theta.append(updated_theta)
    return opt_theta


In [82]:
def Logistic_Regression(X, y, alpha, theta, iterations):
    m = len(y)
    for i in range(iterations):
        opt_theta = gradient_descent(X,y,theta,alpha,m)
        theta = opt_theta
    return theta

In [88]:

initial_theta = [0,0,0,0,0,0,0,0]
alpha = 0.1
iterations = 10000

In [89]:
final_theta = Logistic_Regression(X_train,y_train,alpha,initial_theta,iterations)


In [90]:
final_theta

[2.782986228544027,
 0.3585247001191568,
 -6.835135343154106,
 -2.054051750981707,
 0.486600543583135,
 -29.011275367203528,
 -1.8211726628704537,
 -2.895795707817223]

In [91]:
def score(theta):
    model_score = 0
    length = len(X_test)
    for i in range(length):
        y_prediction = round(hypothesis(theta,X_test[i]))
        y_real = y_test[i]
        if y_prediction == y_real:
            model_score += 1
    score = model_score / length
    return float(score)

In [92]:
print("The model was correct " + str(round((score(final_theta) * 100),3)) + "% of the time.")

The model was correct 71.067% of the time.
